In [17]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1007,  0.0785,  0.2078,  0.0580,  0.1551, -0.0496, -0.2908, -0.1656,
          0.0990,  0.1177],
        [-0.0570,  0.0680,  0.2215, -0.2046,  0.1640, -0.0314, -0.1577, -0.1459,
         -0.0745,  0.1210]], grad_fn=<AddmmBackward0>)

In [18]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [19]:
net = MLP()
net(X)

tensor([[ 3.3395e-02,  1.6892e-01,  9.0341e-02,  5.2319e-02,  8.0197e-02,
         -4.0549e-01,  1.8407e-01,  4.7117e-01, -3.0319e-04,  2.5437e-01],
        [-1.8806e-02,  1.4333e-02,  9.0581e-03, -4.2025e-02,  6.2694e-02,
         -2.2473e-01,  6.4430e-02,  1.5367e-01,  8.3814e-02,  6.9663e-02]],
       grad_fn=<AddmmBackward0>)

In [20]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [21]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1411, -0.1985, -0.2181,  0.1370, -0.2784, -0.0832,  0.2713,  0.0336,
          0.3427, -0.3050],
        [-0.0890, -0.2700, -0.0279, -0.1409, -0.0077, -0.0865,  0.0713,  0.1272,
          0.0978, -0.1784]], grad_fn=<AddmmBackward0>)

In [22]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [23]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0738, grad_fn=<SumBackward0>)

In [24]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2845, grad_fn=<SumBackward0>)

第一题

In [25]:
class MySequential2(nn.Module):
    """顺序块"""
    # 使用list
    def __init__(self, *args):
        super(MySequential2, self).__init__()
        self.sequential = []
        for module in args:
            self.sequential.append(module)

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for module in self.sequential:
            X = module(X)
        return X

In [27]:
linear1 = nn.Linear(20, 256)
relu = nn.ReLU()
linear2 = nn.Linear(256, 10)
net = MySequential(linear1, relu, linear2)
net2 = MySequential2(linear1, relu, linear2)
# 结果一样
print(net(X))
print(net2(X))
# 使用_modules方便打印net的网络结构和参数，而list则无法做到
# print(net, '\n', net.state_dict())
# print(net2, '\n', net2.state_dict())

tensor([[ 0.2199, -0.1385,  0.0916,  0.1277, -0.5018,  0.0185, -0.0627, -0.2026,
          0.0553, -0.0475],
        [ 0.0114, -0.0190, -0.0807,  0.0191, -0.2679, -0.0304, -0.0155, -0.1750,
         -0.1130, -0.1277]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2199, -0.1385,  0.0916,  0.1277, -0.5018,  0.0185, -0.0627, -0.2026,
          0.0553, -0.0475],
        [ 0.0114, -0.0190, -0.0807,  0.0191, -0.2679, -0.0304, -0.0155, -0.1750,
         -0.1130, -0.1277]], grad_fn=<AddmmBackward0>)


第二题

In [28]:
class MyBlock(nn.Module):
    def __init__(self, block1, block2):
        super(MyBlock, self).__init__()
        self.block1 = block1
        self.block2 = block2

    def forward(self, X):
        X = torch.cat((self.block1(X), self.block2(X)), 1)
        return X


block = MyBlock(nn.Linear(20, 5), nn.Linear(20, 5))
net = nn.Sequential(block, nn.ReLU(), nn.Linear(10, 5))
print(net(X).shape)

torch.Size([2, 5])


第三题

In [30]:
class Factory(nn.Module):
    def __init__(self, net, ins, outs, k):
        super().__init__()
        for idx in range(k):
            self._modules[str(idx)] = net(ins, outs)

    def forward(self, X):
        res = list()
        for idx in range(len(self._modules)):
            res.append(self._modules[str(idx)](X))
        return torch.concat(res, 1)

X = torch.rand(2, 20)
f = Factory(nn.Linear, 20, 256, 3)
print(f(X).shape, f(X))
print(f)

torch.Size([2, 768]) tensor([[ 0.1617, -0.1429, -0.4872,  ...,  0.3070, -0.2481,  0.0582],
        [-0.2420, -0.2882, -0.4634,  ...,  0.3515, -0.1927,  0.2556]],
       grad_fn=<CatBackward0>)
Factory(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): Linear(in_features=20, out_features=256, bias=True)
  (2): Linear(in_features=20, out_features=256, bias=True)
)
